# sources:

## audio preparation:
* https://www.tensorflow.org/tutorials/audio/simple_audio
* https://www.tensorflow.org/io/tutorials/audio

## siamese network:
* https://github.com/hlamba28/One-Shot-Learning-with-Siamese-Networks

## misc:
* https://gitlab.tu-berlin.de/dl4aed/dl4aed-lectures/blob/master/04-audio-preprocessing.ipynb

In [ ]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q
python3 -m pip install --no-deps tensorflow-io==0.17.0

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
''' Small hack to restart kernel after netbook ran to free GPU memory '''
## restart jupyter kernel to free all memory in notebook
from IPython.display import display_html
def restartkernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [ ]:
''' Some imports '''
import tensorflow as tf
assert tf.__version__ >= "2.4.0"
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
''' Allow typical dynamic GPU memory allocation and read config'''
from utils import load_config, save_config, allow_growth, update
allow_growth()

# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
config = load_config(verbose=0)

# some other configuration (*.py)
import configuration

In [ ]:
''' Parsing CLI arguments and overwriting config '''
from argument_parser import parse_arguments
parsed_arguments, _ = parse_arguments()

In [ ]:
# overwrite config
config = update(config, configuration.config)
config = update(config, parsed_arguments.__dict__)

In [ ]:
import os
os.environ["WANDB_PROJECT"] = "training"
os.environ["WANDB_ENTITY"] = "dl4aed"
os.environ["WANDB_API_KEY"] = "****"
os.environ["WANDB_MODE"] = "run" #or "dryrun"
os.environ["WANDB_NOTEBOOK_NAME"] = "model"

In [ ]:
'''
from wandb_utils import WandbWrapper
wandb_wrapper = WandbWrapper(config)
config = wandb_wrapper.get_config()
''';

In [ ]:
# extract parameter classes
data_parameter = config["data_parameter"]
model_parameter = config["model_parameter"]
training_parameter = config["training_parameter"]

In [ ]:
''' Some small amount of reproducibility '''
tf.random.set_seed(training_parameter["seed"])
np.random.seed(training_parameter["seed"])

In [ ]:
''' Create model and save config '''
from network import Network
network = Network(model_parameter, training_parameter)
network.compile()
network.save()
# save config into model path
save_config(data_parameter, model_parameter, training_parameter, network)

In [ ]:
config = {}
config['data_root'] = 'gtzan'
config['noise_path'] = '/media/datasets/fsdkaggle2018/FSDKaggle2018.meta/test_post_competition_scoring_clips.csv'
config['sample_rate'] = 22050
config['nfft'] = 512
config['window'] = 512
config['stride'] = 256
config['mels'] = 64
config['fmin_mels'] = 0
config['fmax_mels'] = 8000
config['time_mask'] = 10
config['freq_mask'] = 10
config['noise_threshold'] = 1 # add noise to only 0.3
config['beta'] = 0.5 # noise strength when mixing mel spectrograms
config['SNR'] = 5
config['noise_root'] = '/media/datasets/fsdkaggle2018/FSDKaggle2018.audio_test/'
config['shuffle_batch_size'] = 64

from preprocessor import Preprocessor
y = Preprocessor(config=config)
y.create_logger()

y.set_config({'fade': 10000,
              'epsilon': 0.1,
              'roll_val': 15,
              'top_db': 80,
              'shift_val': 3,
              'bins_per_octave': 12,
              'param_db': 10,
              'train_size': 0.7,
              'val_size': 0.2,
              'test_size': 0.1,
              'noisy_samples': 5})

y.load_data(data_dir="/media/datasets/tfds/")

for mode in y.available_modi:
    y.offline_preprocessing(mode)
y.save_mels()

train_ds = y.train_ds
val_ds = y.val_ds
test_ds = y.test_ds

In [ ]:
''' Training procedure '''
AUTOTUNE = tf.data.AUTOTUNE

network.fit(train_ds, epochs=training_parameter["epochs"],
                   initial_epoch=network.epoch, max_queue_size=AUTOTUNE,
                   workers=AUTOTUNE, use_multiprocessing=False, callbacks=network.callbacks)# + [wandb_wrapper.get_callback()])

In [ ]:
''' Evaluation procedure '''
#network.evaluate(test_dataset)

In [ ]:
# restart kernel to free GPU mem
restartkernel()